In [1]:
from src.stats_model import *
import scipy.linalg as la
import numpy as np

In [79]:
def find_steady_iso(T, svd_cutoff=1E-15, tol=1E-15):
    US = None
    step = 0
    while True:

        # get SVD
        l, r, u = T.shape
        M = T.reshape((l, r*u))
        U, S, Vh = np.linalg.svd(M)
        
        '''
        # make sure size is same
        if US is not None:
            _, usr = US.shape
            S = S/np.sqrt(np.sum(S**2))
            S = S[0:usr]
            U = U[:, 0:usr]
        '''

        # normalize singular values
        S = S/np.sqrt(np.sum(S**2))

        # cut singular values
        #S = S[S > svd_cutoff]
        #Vh = Vh[0:S.shape[0], :]
        #U = U[:, 0:S.shape[0]]
        new_US = U @ np.diag(S)

        # calculate diff
        if US is None or np.any(new_US.shape != US.shape):
            diff = np.Inf
        else:
            diff = np.max(abs(new_US - US))
        if step % 100 == 0:
            print(f"iso_step {step}; iso_diff = {diff}")

        # return or go to new loop
        if diff < tol:
            num_S = S[S > svd_cutoff].shape[0]
            vl, vr = Vh.shape
            Vh = Vh.reshape((vl, vr//u, u))
            return Vh[0:num_S, 0:num_S]
        else:
            US = new_US
            T = np.einsum("ijk,jl->ilk", T, US)
            T = T/np.sqrt(np.sum(T**2))
            step += 1

def find_steady_mps_site(mps_site, mpo_site, svd_cutoff=1E-15, tol=1E-15):
    l, m, n, _ = mpo_site.shape

    step = 0
    ms = find_steady_iso(mps_site, svd_cutoff, tol)
    while True:
        i, j, _ = ms.shape
        new_ms = np.einsum("ijk,lmnk->iljmn", ms, mpo_site)
        new_ms = new_ms.reshape((i*l, j*m, n))
        new_ms = find_steady_iso(new_ms, svd_cutoff, tol)

        if new_ms.shape != ms.shape:
            diff = np.Inf
        else:
            diff = np.max(abs(new_ms - ms))
        if step % 1 == 0:
            print(f"mps_step {step}; mps_step = {diff}")
        
        if diff < tol:
            return new_ms
        else:
            ms = new_ms
            step += 1


d = 2
mu = 1

T = SM_TI_site(d, mu, "down")
M = SM_TI_site(d, mu, "open")
find_steady_mps_site(T, M, 1E-10, 1E-10)

iso_step 0; iso_diff = inf
iso_step 100; iso_diff = 0.012663225338865205
iso_step 200; iso_diff = 0.0005243098354590345
iso_step 300; iso_diff = 0.00021578902558958148
iso_step 400; iso_diff = 0.00021389967357757816
iso_step 500; iso_diff = 0.00034580915530697614
iso_step 600; iso_diff = 0.0001628359278028903
iso_step 700; iso_diff = 5.754702418857378e-05
iso_step 800; iso_diff = 2.9643693258432276e-05
iso_step 900; iso_diff = 2.1369577968287574e-05
iso_step 1000; iso_diff = 1.4114181347637752e-05
iso_step 1100; iso_diff = 8.62244793564274e-06
iso_step 1200; iso_diff = 5.287206151371103e-06
iso_step 1300; iso_diff = 3.024168821729429e-06
iso_step 1400; iso_diff = 1.7500070566677423e-06
iso_step 1500; iso_diff = 1.1055244172348572e-06
iso_step 1600; iso_diff = 6.683631400496197e-07
iso_step 1700; iso_diff = 3.9839441144102975e-07
iso_step 1800; iso_diff = 2.3976592669173e-07
iso_step 1900; iso_diff = 1.454078022682939e-07
iso_step 2000; iso_diff = 8.757795582123849e-08
iso_step 2100; is

KeyboardInterrupt: 

In [90]:
def largest_eigvec(M):
    w, vr = la.eig(M)
    v = vr[:, np.argmax(w)]
    return v

d = 10000
mu = 1

T = SM_TI_site(d, mu)

L = delta(7, 3)
stop = False
while not stop:
    M1 = np.einsum("ijk,opkn,lmn->ioljpm", L, T, L).reshape((7**3, 7**3))
    E = largest_eigvec(M1).reshape((7, 7, 7)).swapaxes(0, 1)

    M2 = np.einsum("ijk,ilm->jlkm", E, E).reshape((7**2, 7**2))
    C = largest_eigvec(M2).reshape((7, 7))
                                
    M3 = np.einsum("ijk,ilop,lmn->jomkpn", E, T, E).reshape((7**3, 7**3))
    LC = largest_eigvec(M3).reshape((7, 7, 7))
    new_L = np.einsum("ijk,kl->ijl", LC, la.pinv(C)).swapaxes(1, 2)
    diff = np.sum(abs(new_L - L))
    print(diff)
    if diff < 1E-5:
        stop = True
    L = new_L

6.010127031412342
3.04643613230867e-10


In [91]:
np.nonzero(L)

(array([5, 5, 5, 5, 5, 5, 5], dtype=int64),
 array([5, 5, 5, 5, 5, 5, 5], dtype=int64),
 array([0, 1, 2, 3, 4, 5, 6], dtype=int64))

In [93]:
L[5, 5, :]

array([1.25078154e-07+0.j, 5.00100020e-03+0.j, 3.75050018e-05+0.j,
       3.75050018e-05+0.j, 2.50098471e-05+0.j, 9.99974989e-01+0.j,
       5.00087518e-03+0.j])

In [94]:
C

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])